In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

cricket_match = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "cricket_match") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

cricket_match.show()

In [ ]:
spark.sql('''
with all_matches as (
select
    cricket_match.batter as player,
    cricket_match.match_id as match_id
from
    cricket_match
union all
select
    cricket_match.bowler as player,
    cricket_match.match_id as match_id
from
    cricket_match),
total_matches as (
select
    player,
    count(distinct match_id) as total_matches
from
    all_matches
group by
    player),
batting_matches as (
select
    batter,
    count(distinct match_id) as batting_matches
from
    cricket_match
group by batter),
bowling_matches as (
select
    bowler,
    count(distinct match_id) as bowling_matches
from
    cricket_match
group by bowler)
select
    total_matches.player,
    total_matches,
    ifnull(batting_matches,0) as batting_matches,
    ifnull(bowling_matches,0) as bowling_matches
from
    total_matches
left join
    batting_matches
on
    total_matches.player = batting_matches.batter
left join
    bowling_matches
on
    total_matches.player = bowling_matches.bowler
''').show()